In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("/Users/elmiraonagh/Desktop/courses/6444/project/SWE-manger/SWE-Manger/Project/RQ2/SWE_bench_lite_with_resolution.csv")
df.head()

,repo,instance_id,version,n_files_touched,patch_size,n_test_files_touched,test_patch_size,num_FAIL_TO_PASS,problem_length_words,predicted_GH_labels,...,ExpeRepair_Claude4,Refact_ai_Agent,KGCompass_Claude4,SWE-agent_Claude4,SWE-agent_Claude3_7,KGCompass_DeepSeek_V3,Moatless_Tools_Deepseek_V3,SWE-Fixer_Qwen25-7b,SWE-agent_GPT_4o,SWE-agent_Claude3Opus
0,astropy/astropy,astropy__astropy-12907,4.3,1,2,1,19,2,158,bug,...,1,1,1,1,1,1,0,0,0,1
1,astropy/astropy,astropy__astropy-14182,5.1,1,41,1,28,1,189,enhancement,...,0,0,0,0,0,0,0,0,0,0
2,astropy/astropy,astropy__astropy-14365,5.1,1,4,1,15,1,202,bug,...,0,0,0,0,0,0,0,0,0,0
3,astropy/astropy,astropy__astropy-14995,5.2,1,4,1,39,1,357,bug,...,1,1,1,1,1,1,1,1,1,1
4,astropy/astropy,astropy__astropy-6938,1.3,1,2,2,17,2,76,bug,...,1,1,1,1,0,1,0,1,0,0


In [5]:
# List of agent columns
agents = [
    "ExpeRepair_Claude4",
    "Refact_ai_Agent",
    "KGCompass_Claude4",
    "SWE-agent_Claude4",
    "SWE-agent_Claude3_7",
    "KGCompass_DeepSeek_V3",
    "Moatless_Tools_Deepseek_V3",
    "SWE-Fixer_Qwen25-7b",
    "SWE-agent_GPT_4o",
    "SWE-agent_Claude3Opus"
]

# Compute success rate per agent per cluster
clustered_success = df.groupby('hdbscan_topic_all')[agents].mean().reset_index()
clustered_success

,hdbscan_topic_all,ExpeRepair_Claude4,Refact_ai_Agent,KGCompass_Claude4,SWE-agent_Claude4,SWE-agent_Claude3_7,KGCompass_DeepSeek_V3,Moatless_Tools_Deepseek_V3,SWE-Fixer_Qwen25-7b,SWE-agent_GPT_4o,SWE-agent_Claude3Opus
0,-1,0.700000,0.300000,0.400000,0.600000,0.500000,0.300000,0.200000,0.400000,0.000000,0.000000
1,0,0.716667,0.650000,0.616667,0.666667,0.633333,0.416667,0.516667,0.333333,0.333333,0.233333
2,1,0.625000,0.687500,0.625000,0.562500,0.375000,0.437500,0.187500,0.250000,0.062500,0.062500
3,2,0.576159,0.622517,0.596026,0.576159,0.470199,0.397351,0.271523,0.238411,0.178808,0.105960
4,3,0.354839,0.387097,0.354839,0.354839,0.322581,0.129032,0.193548,0.129032,0.096774,0.064516
5,4,0.718750,0.656250,0.718750,0.531250,0.437500,0.343750,0.281250,0.187500,0.125000,0.062500


In [9]:
# Melt to long format: cluster, agent, success_rate
clustered_success_long = clustered_success.melt(
    id_vars='hdbscan_topic_all', 
    value_vars=agents, 
    var_name='agent', 
    value_name='success_rate'
)
print(len(clustered_success_long))
clustered_success_long.head()

60


,hdbscan_topic_all,agent,success_rate
0,-1,ExpeRepair_Claude4,0.700000
1,0,ExpeRepair_Claude4,0.716667
2,1,ExpeRepair_Claude4,0.625000
3,2,ExpeRepair_Claude4,0.576159
4,3,ExpeRepair_Claude4,0.354839


In [11]:
# Sort by cluster and descending success rate
clustered_success_long = clustered_success_long.sort_values(
    by=['hdbscan_topic_all', 'success_rate'], 
    ascending=[True, False]
).reset_index(drop=True)
clustered_success_long.head(10)

,hdbscan_topic_all,agent,success_rate
0,-1,ExpeRepair_Claude4,0.7
1,-1,SWE-agent_Claude4,0.6
2,-1,SWE-agent_Claude3_7,0.5
3,-1,KGCompass_Claude4,0.4
4,-1,SWE-Fixer_Qwen25-7b,0.4
5,-1,Refact_ai_Agent,0.3
6,-1,KGCompass_DeepSeek_V3,0.3
7,-1,Moatless_Tools_Deepseek_V3,0.2
8,-1,SWE-agent_GPT_4o,0.0
9,-1,SWE-agent_Claude3Opus,0.0


In [12]:
clustered_success_long.to_csv("/Users/elmiraonagh/Desktop/courses/6444/project/SWE-manger/SWE-Manger/Project/RQ2/agent_success_rate.csv", index = False)